In [14]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [15]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [16]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [17]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [18]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [19]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [6]:
## 
n='Input_C_049' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_C_049
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.0002282847510044541
test_rmse: 0.0002349343758849067
test_r2: 0.32313787125456866
feature number: 3
train_rmse: 0.00022439324806441548
test_rmse: 0.000232197878999025
test_r2: 0.33867473029300127
feature number: 4
train_rmse: 0.00022383920870660238
test_rmse: 0.0002324506634396871
test_r2: 0.33643177426273485
feature number: 5
train_rmse: 0.00022352483231455415
test_rmse: 0.00023362770250145044
test_r2: 0.32950338559810116
feature number: 6
train_rmse: 0.00022339003469780465
test_rmse: 0.00023453063748588287
test_r2: 0.32428590882846786
feature number: 7
train_rmse: 0.0002221106716321732
test_rmse: 0.0002341951879991489
test_r2: 0.32654430812438795
feature number: 8
train_rmse: 0.00022172789652128146
test_rmse: 0.00023546602040798215
test_r2: 0.31903375099794945
feature number: 9
train_rmse: 0.00022155419746992385
test_rmse: 0.0002363360861404254
test_r2: 0.31423991900629683
feature number: 10
train_rmse: 0.000220791

train_rmse: 0.00021391845955778563
test_rmse: 0.000230727736750025
test_r2: 0.3450304932181995
feature number: 13
train_rmse: 0.00021302897043895707
test_rmse: 0.00023017349809647435
test_r2: 0.3482669710118563
feature number: 14
train_rmse: 0.00021178598354376697
test_rmse: 0.0002300324660958986
test_r2: 0.34921903830082884
feature number: 15
train_rmse: 0.00021069666678484554
test_rmse: 0.000228269970331703
test_r2: 0.35940418093914617
feature number: 16
train_rmse: 0.00021034750338448194
test_rmse: 0.00022893697046330401
test_r2: 0.354181434980401
feature number: 17
train_rmse: 0.0002093761856610267
test_rmse: 0.00022913761736399933
test_r2: 0.35127186973530017
feature number: 18
train_rmse: 0.00020708357064709907
test_rmse: 0.00022941128800356474
test_r2: 0.3450347478431947
feature number: 19
train_rmse: 0.00020692956651801898
test_rmse: 0.00022953108663921555
test_r2: 0.34415449729088754
feature number: 20
train_rmse: 0.00020663275165437107
test_rmse: 0.00022996182107201907
test_r

train_rmse: 0.00022439324806441548
test_rmse: 0.000232197878999025
test_r2: 0.33867473029300127
feature number: 4
train_rmse: 0.00022383920870660238
test_rmse: 0.0002324506634396871
test_r2: 0.33643177426273485
feature number: 5
train_rmse: 0.00022352483231455415
test_rmse: 0.00023362770250145044
test_r2: 0.32950338559810116
feature number: 6
train_rmse: 0.00022339003469780465
test_rmse: 0.00023453063748588287
test_r2: 0.32428590882846786
feature number: 7
train_rmse: 0.0002221106716321732
test_rmse: 0.0002341951879991489
test_r2: 0.32654430812438795
feature number: 8
train_rmse: 0.00021559920591254393
test_rmse: 0.00022670337594564975
test_r2: 0.3683423198552228
feature number: 9
train_rmse: 0.0002152549302728991
test_rmse: 0.00022834147604245468
test_r2: 0.35903792325915485
feature number: 10
train_rmse: 0.00021419271616068787
test_rmse: 0.00022901862705377525
test_r2: 0.3542289183114134
feature number: 11
train_rmse: 0.0002140517887774573
test_rmse: 0.0002299009820770753
test_r2: 0.

# 可以修改第一層的神經元數量

In [20]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(3, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [21]:
fea_num=[3,4,8,10] # 要測試的feature數量 神經元3

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_2/Input_C_049.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 3
test_rmse_ave: 0.0045989640680223366
[0.0002445873506658554, 0.00039411269014051804, 0.014790299717365668, 0.00027180104286936166, 0.00022294126424278297, 0.00032559173317380115, 0.02471153837537956, 0.0002166571845489782, 0.004442671952300791, 0.0003694393695360495]


test_r2_ave: -1202.5962700343425
[0.2760848663190365, -0.5510978950716479, -3980.8124707072284, 0.23714328574567656, 0.3930878031145598, -0.28484674691115774, -7754.0089986329, 0.09733040685410355, -290.5770371561808, -0.7318955671656864]


feature number: 4
test_rmse_ave: 0.0019012556445273919
[0.01071627550749838, 0.00036034263815869807, 0.00021819046408391206, 0.0003893541504832805, 0.0012813656489498433, 0.00024964122484526154, 0.0002655319699334965, 0.001971555574298979, 0.003287207213172399, 0.00027309205384967126]


test_r2_ave: -164.04108230027379
[-1388.6563332227904, -0.2966706417121536, 0.13344050582556566, -0.5654181062500112, -19.048925435125664, 0.24466919052437186, 0.10460147234662609, -7